In [1]:

# Setup the Jupyter version of Dash
import dash
from django_plotly_dash import DjangoDash
from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash
from bson.json_util import dumps, JSONOptions

import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
import urllib.parse
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import re

#import crud module
from pymongo import MongoClient
from bson.objectid import ObjectId

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import our Crud python file
from myCrud import AnimalShelter


###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "pass"
shelter = AnimalShelter(password, username)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.recordSearch({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
##df.drop(columns=["_id"], inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_name = 'Grazioso Salvare Logo.png' #create variable for our logo
encoded_image = base64.b64encode(open(image_name, 'rb').read()) # encode image with base64

#dash_table
app.layout = html.Div([
    #create html tag for our image
    #The image will take user to "www.snhu.edu"
    html.A([
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
               height = 250, width = 250))], href = 'https://www.snhu.edu', target = "_blank"),
    #center text for website
    html.Center(html.B(html.H1('Zachary Huffs Project 2 CS-340 Dashboard'))),
    html.Hr(),
    #create our radio buttons for the different options
    dcc.RadioItems(
        id='filter-type',
        options=[
            {"label": html.Div(['Water Rescue'], style={'color': 'Blue', 'font-size': 15}), 'value': 'Water'},
            {"label": html.Div(['Mountain Rescue '], style={'color': 'Grey', 'font-size': 15}), 'value': 'Mountain'},
            {"label": html.Div(['Disaster Rescue'], style={'color': 'Red', 'font-size': 15}), 'value': 'Disaster'},
            {"label": html.Div(['Reset'], style={'color': 'Black', 'font-size': 15}), 'value': 'Reset'},
        ],
        inline=True, #puts all buttons on same row
        value='Water'
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", #allow single row to be selected
        selected_rows=[],
        filter_action="native", #allow a filter
        sort_action="native", #allow sorting
        page_action="native",
        page_current=0, #start page
        page_size=10, 

    ),
    html.Br(),
    html.Hr(),
#This will be the location for our chart, and map 
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content':'center'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])
      
# Interaction Between Buttons and Backend


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(user_choice, **kwargs):
    #set up an if/else if/else block to respond to the radio buttons
    if user_choice == 'Water':
        
        #import regex to make strings look better: "https://colab.research.google.com/github/alvinntnu/python-notes/blob/master/python-basics/regex.ipynb"
        #This library was found online and helped me to sort the strings
        labRegex = re.compile(".*lab.*", re.IGNORECASE)
        chesaRegex = re.compile(".*chesa.*", re.IGNORECASE)
        newRegex = re.compile(".*newf.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.recordSearch({
            '$or':[ 
                {"breed": {'$regex': newRegex}},
                {"breed": {'$regex': chesaRegex}},
                {"breed": {'$regex': labRegex}},
            ],
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    elif user_choice == 'Mountain':
        
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        alaskanRegex = re.compile(".*mala.*", re.IGNORECASE)
        oldRegex = re.compile(".*old engilish.*", re.IGNORECASE)
        huskyRegex = re.compile(".*husk.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.recordSearch({
            '$or':[
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': alaskanRegex}},
                {"breed": {'$regex': oldRegex}},
                {"breed": {'$regex': huskyRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    elif user_choice == 'Disaster':
        
        germanRegex = re.compile(".*german.*", re.IGNORECASE)
        goldenRegex = re.compile(".*golden.*", re.IGNORECASE)
        bloodRegex = re.compile(".*blood.*", re.IGNORECASE)
        doberRegex = re.compile(".*dober.*", re.IGNORECASE)
        rottRegex = re.compile(".*rott.*", re.IGNORECASE)
        
        df = pd.DataFrame.from_records(shelter.recordSearch({
            '$or':[
                {"breed": {'$regex': germanRegex}},
                {"breed": {'$regex': goldenRegex}},
                {"breed": {'$regex': bloodRegex}},
                {"breed": {'$regex': doberRegex}},
                {"breed": {'$regex': rottRegex}},
            ],
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))
        
    elif user_choice == 'Reset':
        df = pd.DataFrame.from_records(shelter.recordSearch({})) 
    else:
        raise Exception("Unknown filter")
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')        
        
    return (data,columns)


#change the color of a selected cell
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns, **kwargs):
    return [{
        'if': { 'column_id': i },
        'background_color': 'White'
    } for i in selected_columns]

#call back for pie chart
#set to plot all of the data across all of the pages instead of the viewable data
#change to derived_viewport_data if other behavior is wanted
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData, **kwargs):
    
    #Creating our Pie Chart for dashboard
    
    dffPie = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(            
            figure = px.pie(dffPie, names='breed',)
        )    
    ]

#call back for slecting a row and then plotting the geomarker
@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(virtualRows, **kwargs):
    
    #Create our view for map pop up, default
    if not virtualRows:
        location_array = (30.75,-97.48) #default marker at Austin Animal Shelter
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
    else: #location if you have specified animal selected
        dff = pd.DataFrame(df.iloc[virtualRows])
        lat = float(dff['location_lat'].to_string().split()[1])
        long = float(dff['location_long'].to_string().split()[1])
        location_array = (lat, long) #build the array based on selection
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    #return the map with specified locations 
    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=location_array,
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=location_array, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]
app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
